# Introduction

Author: Austin Dibble

A short notebook which demonstrates the performance of the models on OMS2CD when they had only been trained on other datasets (such as OSCD, OMCD, and LEVIR-CD).

# Clone repository and download dataset

In [3]:
# from IPython.display import clear_output; key = input(); clear_output()
!git clone https://github.com/Dibz15/OpenMineChangeDetection.git
import sys
sys.path.append('/content/OpenMineChangeDetection/')

Cloning into 'OpenMineChangeDetection'...
remote: Enumerating objects: 3454, done.
remote: Counting objects: 100% (53/53), done.
remote: Compressing objects: 100% (37/37), done.
remote: Total 3454 (delta 23), reused 35 (delta 16), pack-reused 3401
Receiving objects: 100% (3454/3454), 112.52 MiB | 18.00 MiB/s, done.
Resolving deltas: 100% (2247/2247), done.
Updating files: 100% (2404/2404), done.


In [ ]:
from OpenMineChangeDetection.utils import download_prep_oms2cd
download_prep_oms2cd(output_dir='OMS2CD')

# Install dependencies

In [4]:
%%capture
!pip install rasterio==1.3.8
!pip install torchgeo==0.4.1
!pip install tiler==0.5.7
!pip install kornia==0.6.12
!pip install lightning==1.9.5
!pip install torchmetrics==0.11.4

# Seed random, set device

In [5]:
import os
import torch
import random
import numpy as np

if torch.cuda.is_available():
  device = torch.device('cuda')
else:
  device = torch.device('cpu')

torch.manual_seed(543)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(543)
random.seed(543)
np.random.seed(543)

# Load pre-trained models

In [14]:
import shutil
import os
os.makedirs('diffusion_weights', exist_ok=True)
shutil.copyfile('drive/MyDrive/2023_dissertation/model_weights/ddpm_cd/best_cd_model_gen.pth', 'diffusion_weights/best_cd_model_gen.pth')
shutil.copyfile('drive/MyDrive/2023_dissertation/model_weights/LSNet/LSNet_diffFPN_state.pt', 'LSNet_diffFPN_state.pt')

'LSNet_diffFPN_state.pt'

In [19]:
import torch

def load_tinycd(device):
    from OpenMineChangeDetection.TinyCD.models.cd_lightning import ChangeDetectorLightningModule
    tinycd = ChangeDetectorLightningModule(freeze_backbone=False)
    tinycd.change_detector.load_state_dict(torch.load("OpenMineChangeDetection/TinyCD/pretrained_models/levir_best.pth", map_location=device))
    return tinycd.to(device).eval()

def load_lsnet(device):
    from OpenMineChangeDetection.LSNet import LSNetLightning
    from OpenMineChangeDetection.LSNet.utils.parser import get_parser_with_args
    class AttributeDict(object):
        def __init__(self, dictionary):
            for key, value in dictionary.items():
                setattr(self, key, value)

    state = torch.load("LSNet_diffFPN_state.pt", map_location=device)
    opt = get_parser_with_args(metadata_json='OpenMineChangeDetection/LSNet/metadata.json')
    opt = AttributeDict(opt)
    model = LSNetLightning(opt)
    model.model.load_state_dict(state, strict=False)
    return model.to(device).eval()

def load_ddpmcd(device):
    from OpenMineChangeDetection.ddpm_cd.core import logger as Logger
    from OpenMineChangeDetection.ddpm_cd.ddpm_lightning import CD

    class Args:
        def __init__(self):
            self.config = 'OpenMineChangeDetection/ddpm_cd/config/oms2cd.json'
            self.phase = 'train'
            self.gpu_ids = '0'
            self.debug = False
            self.enable_wandb = False
            self.log_eval = False

    opt = Logger.parse(Args())
    opt = Logger.dict_to_nonedict(opt)
    change_detection = CD(opt)
    opt['path_cd']['finetune_path'] = 'diffusion_weights'
    opt['path_cd']['resume_opt'] = None

    if opt['path_cd']['finetune_path'] is not None:
        change_detection.load_network()
    return change_detection.to(device).eval()


In [12]:
tinycd = load_tinycd(device)

In [16]:
lsnet = load_lsnet(device)

In [20]:
ddpmcd = load_ddpmcd(device)

export CUDA_VISIBLE_DEVICES=0


Verifying hashes: Good.


Verifying hashes: Good.
Loading CD gen state dict from diffusion_weights/best_cd_model_gen.pth.


# Prepare dataloader

In [21]:
from OpenMineChangeDetection.datasets import OMS2CDDataModule
from torchgeo.transforms import AugmentationSequential
import kornia.augmentation as K
from OpenMineChangeDetection.ddpm_cd.ddpm_lightning import CD

datamodule = OMS2CDDataModule(root='OMS2CD', bands='rgb', load_area_mask=False,
                              batch_size=1, tile_mode="constant", index_no_mask=True, stride=100)
datamodule.setup('fit')
datamodule.setup('validate')
datamodule.setup('test')

# Evaluate pre-trained models

In [22]:
from OpenMineChangeDetection.utils import evaluate_model, get_mask_preds_tinycd, get_mask_preds_lsnet, get_mask_preds_ddpmcd

stats = evaluate_model(tinycd, datamodule.val_dataloader(), get_mask_preds_tinycd, device, threshold=0.3)
print("Validation stats:")
print(stats)

100%|██████████| 463/463 [00:28<00:00, 16.53it/s]

Validation stats:
{'OA': 0.9863958954811096, 'F1': 0.0003632480220403522, 'recall': 0.00018165743676945567, 'precision': 0.9868420958518982, 'AP': 0.015304083935916424, 'PRC': (tensor([0.0136, 0.3937, 0.5878, 0.6675, 0.7186, 0.7462, 0.7885, 0.8080, 0.8349,
        0.8667, 0.8775, 0.8896, 0.8993, 0.9600, 0.9651, 0.9859, 0.9825, 1.0000,
        1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 1.0000], device='cuda:0'), tensor([1.0000e+00, 2.6183e-03, 1.6858e-03, 1.3128e-03, 1.1069e-03, 9.3977e-04,
        8.3078e-04, 7.3390e-04, 6.4912e-04, 5.3528e-04, 4.3356e-04, 3.5120e-04,
        3.0276e-04, 2.3252e-04, 2.0103e-04, 1.6955e-04, 1.3564e-04, 1.1142e-04,
        7.2663e-05, 5.0864e-05, 4.1176e-05, 3.8754e-05, 2.6643e-05, 1.6955e-05,
        1.2110e-05, 2.4221e-

In [23]:
stats = evaluate_model(tinycd, datamodule.test_dataloader(), get_mask_preds_tinycd, device, threshold=0.3)
print("Test stats:")
print(stats)

100%|██████████| 342/342 [00:15<00:00, 22.18it/s]


Test stats:
{'OA': 0.9820842146873474, 'F1': 0.0, 'recall': 0.0, 'precision': 0.0, 'AP': 0.015359574928879738, 'PRC': (tensor([1.7912e-02, 1.0500e-02, 7.2780e-04, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 0.0000e+00, 1.0000e+00], device='cuda:0'), tensor([1.0000e+00, 8.4687e-05, 2.4908e-06, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        0.0000e+00

In [24]:
stats = evaluate_model(lsnet, datamodule.val_dataloader(), get_mask_preds_lsnet, device, threshold=0.3)
print("Validation stats:")
print(stats)

100%|██████████| 463/463 [00:25<00:00, 18.44it/s]


Validation stats:
{'OA': 0.9863935112953186, 'F1': 0.0, 'recall': 0.0, 'precision': 0.0, 'AP': 0.013590898364782333, 'PRC': (tensor([0.0136, 0.0092, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 1.0000], device='cuda:0'), tensor([1.0000e+00, 2.4221e-05, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,


In [25]:
stats = evaluate_model(lsnet, datamodule.test_dataloader(), get_mask_preds_lsnet, device, threshold=0.3)
print("Test stats:")
print(stats)

100%|██████████| 342/342 [00:19<00:00, 17.13it/s]

Test stats:
{'OA': 0.9820874929428101, 'F1': 0.0, 'recall': 0.0, 'precision': 0.0, 'AP': 0.01782796159386635, 'PRC': (tensor([0.0179, 0.0094, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 1.0000], device='cuda:0'), tensor([1.0000e+00, 1.9926e-05, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
       

In [26]:
stats = evaluate_model(ddpmcd, datamodule.val_dataloader(), get_mask_preds_ddpmcd, device, threshold=0.3)
print("Validation stats:")
print(stats)

100%|██████████| 463/463 [11:01<00:00,  1.43s/it]

Validation stats:
{'OA': 0.9863342642784119, 'F1': 0.00022181823442224413, 'recall': 0.00011141656432300806, 'precision': 0.024351509287953377, 'AP': 0.009571156464517117, 'PRC': (tensor([0.0136, 0.0078, 0.0067, 0.0065, 0.0068, 0.0071, 0.0077, 0.0087, 0.0101,
        0.0110, 0.0136, 0.0155, 0.0182, 0.0218, 0.0224, 0.0264, 0.0348, 0.0369,
        0.0459, 0.0300, 0.0162, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 1.0000], device='cuda:0'), tensor([1.0000e+00, 1.6020e-02, 6.8885e-03, 3.7664e-03, 2.3155e-03, 1.4557e-03,
        9.8822e-04, 7.2663e-04, 5.6193e-04, 4.1176e-04, 3.4394e-04, 2.7128e-04,
        2.2526e-04, 1.8650e-04, 1.3322e-04, 1.0899e-04, 9.9306e-05, 6.7819e-05,
        5.3286e-05, 2.1799e-05, 7.2663e-06, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 0.000

In [27]:
stats = evaluate_model(ddpmcd, datamodule.test_dataloader(), get_mask_preds_ddpmcd, device, threshold=0.3)
print("Test stats:")
print(stats)

100%|██████████| 342/342 [08:09<00:00,  1.43s/it]

Test stats:
{'OA': 0.9820408225059509, 'F1': 0.0004966168198734522, 'recall': 0.00024907966144382954, 'precision': 0.0801924616098404, 'AP': 0.01696985960006714, 'PRC': (tensor([0.0179, 0.0178, 0.0185, 0.0197, 0.0217, 0.0238, 0.0256, 0.0292, 0.0341,
        0.0390, 0.0421, 0.0465, 0.0515, 0.0590, 0.0720, 0.0838, 0.0943, 0.1149,
        0.1406, 0.1277, 0.1622, 0.1951, 0.2692, 0.2500, 0.2000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 1.0000], device='cuda:0'), tensor([1.0000e+00, 2.6126e-02, 1.3463e-02, 7.9880e-03, 5.1012e-03, 3.4024e-03,
        2.3090e-03, 1.6863e-03, 1.2977e-03, 1.0138e-03, 7.4973e-04, 5.7786e-04,
        4.4585e-04, 3.5867e-04, 2.9890e-04, 2.2915e-04, 1.7436e-04, 1.2703e-04,
        8.9669e-05, 4.4834e-05, 2.9890e-05, 1.9926e-05, 1.7436e-05, 9.9632e-06,
        4.9816e-06, 0.0000e+00, 0.0